In [1]:
!python --version
!cat /etc/os-release
!cat /usr/local/spark/python/pyspark/version.py

Python 3.8.5
NAME="Ubuntu"
VERSION="20.04 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal
__version__='3.0.0'


In [2]:
from pyspark.sql import SparkSession 
import pandas as pd

In [3]:
spark = SparkSession.builder.appName("version_check").master("local").getOrCreate() 
print(spark.sparkContext.version)

3.0.0


In [4]:
spark_file_dir = "/home/jovyan/work/Spark-The-Definitive-Guide"

# Ch 3. 스파크 기능 둘러보기

In [5]:
!cat ~/work/Spark-The-Definitive-Guide/code/A_Gentle_Introduction_to_Spark-Chapter_3_A_Tour_of_Sparks_Toolset.py

staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/data/retail-data/by-day/*.csv")

staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema


# COMMAND ----------

from pyspark.sql.functions import window, column, desc, col
staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .sort(desc("sum(total_cost)"))\
  .show(5)


# COMMAND ----------

streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/data/retail-data/by-day/*.csv")


# COMMAND ----------

purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("Custom

In [6]:
spark = SparkSession.builder.appName("ch3").master("local").getOrCreate()

In [7]:
spark

In [8]:
spark.conf.set("spark.sql.shuffle.partitions", "5") # partitions(데이터 분할) 200(default) -> 5

In [9]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load(spark_file_dir + "/data/retail-data/by-day/*.csv")

In [10]:
staticDataFrame 

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [11]:
staticDataFrame.createOrReplaceTempView("retail_data")

In [12]:
staticSchema = staticDataFrame.schema

In [13]:
staticSchema

StructType(List(StructField(InvoiceNo,StringType,true),StructField(StockCode,StringType,true),StructField(Description,StringType,true),StructField(Quantity,IntegerType,true),StructField(InvoiceDate,StringType,true),StructField(UnitPrice,DoubleType,true),StructField(CustomerID,DoubleType,true),StructField(Country,StringType,true)))

In [14]:
staticSchema[:2]

StructType(List(StructField(InvoiceNo,StringType,true),StructField(StockCode,StringType,true)))

In [15]:
type(staticSchema)

pyspark.sql.types.StructType

In [16]:
from pyspark.sql.functions import window, column, desc, col

In [17]:
%%time
staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .sort(desc("sum(total_cost)"))\
  .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   17450.0|[2011-09-20 00:00...|          71601.44|
|      null|[2011-11-14 00:00...|          55316.08|
|      null|[2011-11-07 00:00...|          42939.17|
|      null|[2011-03-29 00:00...| 33521.39999999998|
|      null|[2011-12-08 00:00...|31975.590000000007|
+----------+--------------------+------------------+
only showing top 5 rows

CPU times: user 7.89 ms, sys: 8.65 ms, total: 16.5 ms
Wall time: 17.4 s


In [18]:
%%time
staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .sort(desc("sum(total_cost)"))\
  .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   17450.0|[2011-09-20 00:00...|          71601.44|
|      null|[2011-11-14 00:00...|          55316.08|
|      null|[2011-11-07 00:00...|          42939.17|
|      null|[2011-03-29 00:00...| 33521.39999999998|
|      null|[2011-12-08 00:00...|31975.590000000007|
+----------+--------------------+------------------+
only showing top 5 rows

CPU times: user 10.3 ms, sys: 10.3 ms, total: 20.7 ms
Wall time: 15.4 s


In [19]:
%%time
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/data/retail-data/by-day/*.csv")

CPU times: user 10.1 ms, sys: 0 ns, total: 10.1 ms
Wall time: 51.8 ms


In [20]:
streamingDataFrame

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [21]:
streamingDataFrame.isStreaming

True

In [22]:
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost") # lazy evaluation

In [23]:
purchaseByCustomerPerHour

DataFrame[CustomerId: double, window: struct<start:timestamp,end:timestamp>, sum(total_cost): double]

In [24]:
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start() # Action

In [25]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(5)

+----------+------+---------------+
|CustomerId|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+



In [26]:
purchaseByCustomerPerHour.writeStream\
    .format("console")\
    .queryName("customer_purchases_2")\
    .outputMode("complete")\
    .start() # Action

In [27]:
from pyspark.sql import Row

In [28]:
spark.sparkContext.parallelize([Row(1), Row(2), Row(3)]).toDF() # ?????

DataFrame[_1: bigint]

In [29]:
# !cat ~/work/Spark-The-Definitive-Guide/code/A_Gentle_Introduction_to_Spark-Chapter_3_A_Tour_of_Sparks_Toolset.py